In [1]:
import yfinance as yf
import pandas as pd
from tabulate import tabulate

ticker_symbols = ['AAPL', 'AMZN', 'GOOGL', 'META', 'TSLA']
index_symbols = ['SPY', 'QQQ', 'UKX', '^N225', '^GDAXI']
start_date = '2010-01-01'
end_date = '2023-05-01'

stock_data = yf.download(ticker_symbols, start=start_date, end=end_date)['Adj Close']
index_data = yf.download(index_symbols, start=start_date, end=end_date)['Adj Close']

stock_metrics = []
for ticker in ticker_symbols:
    cumulative_returns = (1 + stock_data[ticker].pct_change()).cumprod().iloc[-1]

    rf_rate = 0.01  
    annualized_returns = (cumulative_returns ** (252 / len(stock_data))) - 1
    annualized_volatility = stock_data[ticker].pct_change().std() * (252 ** 0.5)

    downside_returns = stock_data[ticker].pct_change().copy()
    downside_returns[downside_returns > 0] = 0
    downside_volatility = downside_returns.std() * (252 ** 0.5)

    sharpe_ratio = (annualized_returns - rf_rate) / annualized_volatility
    sortino_ratio = (annualized_returns - rf_rate) / downside_volatility

    rolling_max = stock_data[ticker].cummax()
    daily_drawdown = stock_data[ticker] / rolling_max - 1
    max_drawdown = daily_drawdown.cummin().iloc[-1]

    stock_metrics.append([ticker, cumulative_returns, sharpe_ratio, annualized_volatility, sortino_ratio, max_drawdown])

index_metrics = []
for index in index_symbols:
    
    cumulative_returns = (1 + index_data[index].pct_change()).cumprod().iloc[-1]

    rf_rate = 0.01  
    annualized_returns = (cumulative_returns ** (252 / len(index_data))) - 1
    annualized_volatility = index_data[index].pct_change().std() * (252 ** 0.5)

    downside_returns = index_data[index].pct_change().copy()
    downside_returns[downside_returns > 0] = 0
    downside_volatility = downside_returns.std() * (252 ** 0.5)

    sharpe_ratio = (annualized_returns - rf_rate) / annualized_volatility
    sortino_ratio = (annualized_returns - rf_rate) / downside_volatility

    rolling_max = index_data[index].cummax()
    daily_drawdown = index_data[index] / rolling_max - 1
    max_drawdown = daily_drawdown.cummin().iloc[-1]

    index_metrics.append([index, cumulative_returns, sharpe_ratio, annualized_volatility, sortino_ratio, max_drawdown])

stock_df = pd.DataFrame(stock_metrics, columns=['Ticker', 'Cumulative Return', 'Sharpe Ratio', 'Volatility', 'Sortino Ratio', 'Max Drawdown'])
index_df = pd.DataFrame(index_metrics, columns=['Ticker', 'Cumulative Return', 'Sharpe Ratio', 'Volatility', 'Sortino Ratio', 'Max Drawdown'])

stock_df = stock_df.set_index('Ticker').T.reset_index()
index_df = index_df.set_index('Ticker').T.reset_index()

print("Stocks Metrics:")
stock_table = tabulate(stock_df, headers='keys', tablefmt='fancy_grid')
print(stock_table)

print("-" * 80)

print("Indices Metrics:")
index_table = tabulate(index_df, headers='keys', tablefmt='fancy_grid')
print(index_table)


[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  5 of 5 completed
Stocks Metrics:
╒════╤═══════════════════╤═══════════╤═══════════╤═══════════╤═══════════╤════════════╕
│    │ index             │      AAPL │      AMZN │     GOOGL │      META │       TSLA │
╞════╪═══════════════════╪═══════════╪═══════════╪═══════════╪═══════════╪════════════╡
│  0 │ Cumulative Return │ 26.0834   │ 15.7506   │  6.84373  │  6.28616  │ 103.167    │
├────┼───────────────────┼───────────┼───────────┼───────────┼───────────┼────────────┤
│  1 │ Sharpe Ratio      │  0.936217 │  0.663731 │  0.533089 │  0.338834 │   0.709075 │
├────┼───────────────────┼───────────┼───────────┼───────────┼───────────┼────────────┤
│  2 │ Volatility        │  0.286002 │  0.331799 │  0.272976 │  0.407763 │   0.573807 │
├────┼───────────────────┼───────────┼───────────┼───────────┼───────────┼────────────┤
│  3 │ Sortino Ratio     │  1.56389  │  1.1254   │  0.